<a href="https://colab.research.google.com/github/Sparsh9990/Twitter-Sentiment-Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing Kaggle
! pip install Kaggle

In [ ]:
# Configuring Kaggle.Json file path
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
# API to fetch the Dataset from Kaggle
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 83% 67.0M/80.9M [00:00<00:00, 246MB/s]
100% 80.9M/80.9M [00:00<00:00, 227MB/s]


In [ ]:
# Extracting the Dataset from the compressed file
from zipfile import ZipFile
dataset='/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("Data has been extracted")

Data has been extracted


In [ ]:
# Importing Dependencies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# Importing natural language tool kit
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Definition-
Stopwords are words that don't carry much contextual importance when it comes to analysis and aren't required by the ML model hence are removed.

WHY WE DO THIS?
The Dataset is huge so we do our best to remove any noise from the data to reduce its size and complexity. One way to do it is by removing stopwords as they don't add much influential meaning.

In [ ]:
# Printing the Stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# Data Processing
# Converting csv datafile into pandas dataframe
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [ ]:
# Checking dataframe shape to ensure all data was converted
twitter_data.shape

(1599999, 6)

In [ ]:
# Printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


read_csv() function is reading the first data point as the column names and starting count of actual data from the second datapoint so in order to fix that and actually have meaningful column names we make a list of column names and read the dataset again with "names" parameter insside the read_csv() function.

In [ ]:
# Create list of column names and re-read the dataset
column_names  = ['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names, encoding='ISO-8859-1')
twitter_data.shape

(1600000, 6)

In [ ]:
# Printing the first 5 rows of the dataframe
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# Checking and counting any missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
# Check target column distribution. How many labels there are (0 = negative, 2 = neutral, 4 = positive).
# This tells us how many positive, negative and neutral tweets there are in the dataset.
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


Here we find that the data is equally distributed between negative and positive tweets with each being 800000 to make the total dataset to be 16000000. Note- There are no neutral tweets. Also this shows that the data distribution is equal which is optimal for our ML model. If the data is not distributed evenly then our ML model is prone to erroneous results.

In [ ]:
# Converting the Labels (0-negative and 4-positive) into more meaningful (0-negative and 1-positive)
# inplace=True parameter means make the change in the original dataframe. Without this changes wouldn't be stored.
twitter_data.replace({'target':{4:1}},inplace=True)

In [ ]:
# Check target column distribution again to ensure changes were made.
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


**Stemming-** Process of reducing a word to its root word. E.g., words like [acting, actor, actress] get reduced to [act]. This is done to reduce the complexity of the data so that our ML model can quickly process the data and understand whether it's a positive or negative tweet.

We use the PorterStemmer function from nltk library for this that we imported earlier.

In [ ]:
# Calling the PorterStemmer instance in a variable
port_stem = PorterStemmer()

# Creating a function that when called completes the stemming function on every word of every tweet.
def stemming(content):
# Content argument is essentially the text column of the dataframe containing all the tweets.
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
# We do some processing since the ML model understands numbers and "target" label already is in numerical
# form (0 and 1) but the "text" column (Actual tweets) are a mix of alphabetical, numerical and special characters.
# Therefore we use "re" regular expression module to remove (^) anything that doesn't lie in a-z or A-Z from content since
# it doesn't help us determine if a tweet is positive or negative.
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
# Converting all the words into lower case because uppercasing serves no purpose and putting them in a list
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
# .stem() to reduce to root word for faster processing the words that are in the list of stemmed_content and are not in stopwords.
  stemmed_content = ' '.join(stemmed_content)
# Joining all the reduces root words of a tweet together.
  return stemmed_content

In [ ]:
# We create a new column in the dataframe and add all the stemmed data in it.
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [ ]:
# We see the stemmed content column created from text column and now to train the ML model we use the target and stemmed content column.
twitter_data.head()
print(twitter_data['target'], twitter_data['stemmed_content'], sep="\n\n")

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
# Storing targets and tweets in separate variables
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values
print(X,Y, sep="\n\n")

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']

[0 0 0 ... 1 1 1]


In [ ]:
# We split the data into train and test sets using train_test_split() function.

# X_train has tweets and Y_train has corresponding labels to train the model. Similarly,
# X_test has some tweets and Y_test has corresponding labels to evaluate the model.

# test_size=0.2 means that 20% of the data goes for testing while 80% is used in training.

# stratify=Y means we want an equal distribution of the 0 and 1 target labels. So the proportion of positive tweets and negative tweets in the training set is 50-50.
# Its the same in test data set too. If not done, there's a chance all negative tweets might go in test data instead of training data and then our model wouldn't respond
# accurately since it never got trained on the negative tweets. so this parameters ensures we get equal proportion of the 2 labels i.e. 0 and 1 both in training and test datasets.

# random_state=2 This means that every time you run your code with random_state=2, the split will be exactly the same.
# Using the same random_state ensures that the split between your training and testing sets is consistent.
# For example, if you run your code with random_state=2, you will always get the same training and testing sets.
# If someone else runs your code with the same random_state, they will get the same results, making it easier to collaborate and compare findings.
X_train, X_test, Y_train,Y_test= train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape,end="\n\n")
print(X_train, X_test, sep="\n\n")

(1600000,) (1280000,) (320000,)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


Since ML model doesn't understand text, we have to conduct feature extraction inorder to convert this into numerical values using the vectorizer method.


In [ ]:
# TfidfVectorizer() function converts all the words into some numerical values.
# How it does this is it looks at the word, checks how many times it's been repeated in the entire dataset and based on it gives it a value
# Then it tries to establish a link between the value it just assigned and the labels in the dataset. E.g., "happy" word was used 10000 times
# Everytime it was used the label was 0-positive. so the model can get a good idea that whenever this word would be used next  its most likely
# in a positive context.

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train, X_test, sep="\n\n")

  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (1279996, 434014)	0.27189450523324465
  (1279996, 390130)	0.2206474219107611
  (1279996, 373144)	0.35212500999832036
  (1279996, 23807

In [ ]:
# Training the Logistic Regression ML Model
model=LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
# To conduct Model evaluation we use accuracy store on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy Score :', test_data_accuracy)

Accuracy Score : 0.77668125


In [ ]:
# To conduct Model evaluation we use accuracy store on the training data
X_training_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_training_prediction)
print('Accuracy Score :', training_data_accuracy)

Accuracy Score : 0.79871953125


Since the training and test predictions are close enough we say the model has performed well and there's no overfitting.

Overfitting- When training data accuracy is greater than test data accuracy showcasing that the model hasn't learn't properly instead has learnt only what it's been trained on.